# Fine-tuning BAAI/bge-m3 với Dữ liệu Feedback

Notebook này thực hiện quá trình fine-tuning model embedding `BAAI/bge-m3` bằng cách sử dụng bộ dữ liệu triplets (`query`, `positive`, `negative`) đã được tạo từ file `embedding_training_data.jsonl`.

## 1. Cài đặt các thư viện cần thiết

In [ ]:
!pip install -U sentence-transformers datasets

## 2. Tải và chuẩn bị dữ liệu

In [ ]:
from datasets import load_dataset
from sentence_transformers import InputExample
from tqdm.notebook import tqdm

# Tải dữ liệu từ file jsonl
data_file = 'embedding_training_data.jsonl'
dataset = load_dataset('json', data_files=data_file, split='train')

print(f"Đã tải {len(dataset)} mẫu training.")
print("Ví dụ một mẫu:", dataset[0])

Bây giờ, chúng ta cần chuyển đổi dữ liệu thành định dạng `InputExample` mà `sentence-transformers` có thể sử dụng. Đối với `MultipleNegativesRankingLoss`, mỗi `InputExample` sẽ chứa một bộ ba `[query, positive_passage, negative_passage]`.

In [ ]:
train_examples = []
for item in tqdm(dataset, desc="Chuẩn bị dữ liệu"):
    train_examples.append(InputExample(texts=[item['query'], item['pos'], item['neg']]))

print(f"\nĐã tạo {len(train_examples)} InputExample.")
print("Ví dụ một InputExample:")
print(train_examples[0].texts)

## 3. Khởi tạo Model và DataLoader

### Xử lý lỗi không tải được model (Quan trọng)

Lỗi `Can't load the model for 'BAAI/bge-m3'` thường xảy ra do vấn đề về mạng (ví dụ: firewall, proxy trong môi trường công ty) ngăn cản việc tải model trực tiếp từ Hugging Face Hub.

**Giải pháp:** Tải model về máy thủ công và load từ đường dẫn cục bộ.

1.  **Tải model:** Mở terminal và chạy lệnh sau để tải model vào một thư mục có tên `bge-m3-model`:
    ```bash
    git lfs install
    git clone https://huggingface.co/BAAI/bge-m3 bge-m3-model
    ```
    *Lưu ý: Bạn cần cài đặt [Git LFS](https://git-lfs.com) để tải các file model lớn.*

2.  **Chạy cell dưới đây:** Code đã được sửa để load model từ thư mục `bge-m3-model` bạn vừa tải về.

In [ ]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader

# Đường dẫn đến thư mục model đã tải về
# LƯU Ý: Hãy chắc chắn rằng bạn đã chạy lệnh git clone ở trên
# và có một thư mục tên là 'bge-m3-model' trong cùng thư mục với notebook này.
model_name = './bge-m3-model'

# Tải model từ đường dẫn cục bộ
model = SentenceTransformer(model_name)

print("Đã tải xong model BAAI/bge-m3 từ đường dẫn cục bộ.")

## 4. Định nghĩa Loss Function và Training

In [ ]:
# Tạo DataLoader
train_batch_size = 16 # Điều chỉnh tùy theo VRAM của bạn
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

# Sử dụng MultipleNegativesRankingLoss, đây là loss function tiêu chuẩn cho dữ liệu triplets
train_loss = losses.MultipleNegativesRankingLoss(model=model)

# Các tham số training
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up
output_path = './bge-m3-fine-tuned-feedback'

print("Bắt đầu quá trình training...")

### Chuyển sang Vòng lặp Training Thủ công (Để kiểm soát VRAM)

Hàm `model.fit()` rất tiện lợi, nhưng đôi khi có thể gây ra hiện tượng tăng VRAM không kiểm soát (memory leak) trong các phiên training dài. Để khắc phục, chúng ta sẽ tự viết một vòng lặp training thủ công. Cách này cho phép chúng ta kiểm soát hoàn toàn quá trình và đảm bảo bộ nhớ được giải phóng đúng cách sau mỗi bước.

In [ ]:
from torch.optim import AdamW
from tqdm.notebook import trange
import torch

# Định nghĩa optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Bắt đầu training trên thiết bị: {device}")

for epoch in trange(num_epochs, desc="Epoch"):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc="Iteration", smoothing=0.05)
    
    for batch in progress_bar:
        # Chuyển batch sang đúng device
        features, labels = model.batch_to_device(batch, device)
        
        # Tính loss
        loss = train_loss(features, labels)
        
        # Lan truyền ngược và cập nhật trọng số
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # Lấy giá trị loss (dạng số) để theo dõi, tránh memory leak
        current_loss = loss.item()
        total_loss += current_loss
        
        progress_bar.set_description(f"Loss: {current_loss:.4f}")
        
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

print("Training hoàn tất.")

### Lưu lại model đã fine-tuned

In [ ]:
model.save(output_path)
print(f"Model đã được lưu tại: {output_path}")

## 5. Hoàn tất

Quá trình training đã hoàn tất. Model đã được fine-tuned và lưu tại thư mục `bge-m3-fine-tuned-feedback`. Bạn có thể tải và sử dụng model này như một model `sentence-transformer` thông thường.

---

## 6. Đánh giá & Kiểm thử Model (Evaluation & Testing)

Bây giờ, hãy thử sử dụng model vừa fine-tuned để xem nó hoạt động như thế nào với một feedback mới.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Tải model đã fine-tuned
model = SentenceTransformer(output_path)

# Chuẩn bị kho nhãn (corpus) để tìm kiếm
# Chúng ta sẽ lấy tất cả các nhãn duy nhất từ bộ dữ liệu
all_labels = list(set([item['pos'] for item in dataset] + [item['neg'] for item in dataset]))

print(f"Đã tạo kho với {len(all_labels)} nhãn duy nhất.")

# Mã hóa toàn bộ kho nhãn. Chuyển sang GPU nếu có thể.
print("Đang mã hóa kho nhãn...")
corpus_embeddings = model.encode(all_labels, convert_to_tensor=True, show_progress_bar=True)

print("Mã hóa hoàn tất.")

In [ ]:
# Feedback mẫu để kiểm thử
test_query = "App ngân hàng cứ bị văng ra lúc đang chuyển tiền, rất bực mình. Lãi suất gửi tiết kiệm cũng thấp quá."

# Mã hóa feedback
query_embedding = model.encode(test_query, convert_to_tensor=True)

# Sử dụng semantic search để tìm top 5 nhãn tương đồng nhất
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
hits = hits[0] # Lấy kết quả cho query đầu tiên

print(f"Feedback mẫu: '{test_query}'\n")
print("Top 5 nhãn phù hợp nhất:")
for hit in hits:
    print(f"\t{all_labels[hit['corpus_id']]:<40} (Score: {hit['score']:.4f})")

## 7. Đóng gói để Inference

Để tiện sử dụng, chúng ta có thể tạo một lớp đơn giản để đóng gói logic này.

In [ ]:
class FeedbackClassifier:
    def __init__(self, model_path, all_labels):
        self.model = SentenceTransformer(model_path)
        self.labels = all_labels
        print("Đang mã hóa kho nhãn cho classifier...")
        self.label_embeddings = self.model.encode(self.labels, convert_to_tensor=True, show_progress_bar=True)
        print("Classifier sẵn sàng.")
        
    def predict(self, feedback_text, top_k=3):
        query_embedding = self.model.encode(feedback_text, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, self.label_embeddings, top_k=top_k)
        results = []
        for hit in hits[0]:
            results.append({
                'label': self.labels[hit['corpus_id']],
                'score': hit['score']
            })
        return results

# Khởi tạo classifier
classifier = FeedbackClassifier(model_path=output_path, all_labels=all_labels)

# Sử dụng classifier
new_feedback = "Nhân viên tại quầy giao dịch rất nhiệt tình và chuyên nghiệp, nhưng thủ tục mở thẻ tín dụng còn rườm rà."
predictions = classifier.predict(new_feedback, top_k=3)

print(f"\nFeedback: '{new_feedback}'")
print("Dự đoán:")
for pred in predictions:
    print(f"- {pred['label']} (Score: {pred['score']:.4f})")